In [2]:
%tensorflow_version 2.x

In [3]:
!wget https://www.dropbox.com/sh/kpf9z73woodfssv/AAAw1_JIzpuVvwteJCma0xMla?dl=0

--2022-03-07 14:17:55--  https://www.dropbox.com/sh/kpf9z73woodfssv/AAAw1_JIzpuVvwteJCma0xMla?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.3.18, 2620:100:601b:18::a27d:812
Connecting to www.dropbox.com (www.dropbox.com)|162.125.3.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /sh/raw/kpf9z73woodfssv/AAAw1_JIzpuVvwteJCma0xMla [following]
--2022-03-07 14:17:55--  https://www.dropbox.com/sh/raw/kpf9z73woodfssv/AAAw1_JIzpuVvwteJCma0xMla
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc493e01c35b455373dfe88d5118.dl.dropboxusercontent.com/zip_download_get/BEaO6hm_PH0LNZlbmpGuba2YqTdLr4K-TxfQPbRWoztTMQtITnURqlA6RqgxXyKD962VJs7Lmb9rKEdWUh--4l9eY2-NZWxFuDYOT5YuHZsMXQ# [following]
--2022-03-07 14:17:56--  https://uc493e01c35b455373dfe88d5118.dl.dropboxusercontent.com/zip_download_get/BEaO6hm_PH0LNZlbmpGuba2YqTdLr4K-TxfQPbRWoztTMQtITnURqlA6RqgxXyKD962VJs7Lmb9rKEdW

In [4]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from nltk import word_tokenize


In [5]:
filter=pd.read_csv(r'/content/AAAw1_JIzpuVvwteJCma0xMla?dl=0',sep=',', encoding='latin-1')

# New Section

In [ ]:
filter

,PK,question_text,target
0,00002165364db923c7e6,How did Quebec nationalists see their province...,0.0
1,000032939017120e6e44,"Do you have an adopted dog, how would you enco...",0.0
2,0000412ca6e4628ce2cf,Why does velocity affect time? Does velocity a...,0.0
3,000042bf85aa498cd78e,How did Otto von Guericke used the Magdeburg h...,0.0
4,0000455dfa3e01eae3af,Can I convert montra helicon D to a mountain b...,0.0
...,...,...,...
1306119,ffffd48fb36b63db010c,Is foam insulation toxic?,0.0
1306120,ffffec519fa37cf60c78,How can one start a research project based on ...,0.0
1306121,ffffed09fedb5088744a,Who wins in a battle between a Wolverine and a...,0.0
1306122,PKÐÞ¾±´>g´>gPK,NaN,NaN


In [6]:
filter.isnull().sum()

PK            0
question_text    2
target           2
dtype: int64

In [7]:
for col in ['question_text','target']:
  print(col,',',filter[col].value_counts())

question_text , How did Quebec nationalists see their province as a nation in the 1960s?                                      1
What is the best method to teach the pronunciation of English words to kids?                                  1
How much mark should we get to pass in NEET exam?                                                             1
Iâm 13 and overweight. Aside from having a healthy diet, what exercises can I do at home to lose weight?    1
How do you get perfect teeth without using braces?                                                            1
                                                                                                             ..
Why are all psychotherapists so bad at their jobs?                                                            1
How many babies are born each day in India?                                                                   1
What are the milk substitutes for baking a cake?                                        

In [8]:
filter.drop([1306122,1306123],0,inplace=True)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  """Entry point for launching an IPython kernel.


In [9]:
filter.isnull().sum()

PK            0
question_text    0
target           0
dtype: int64

In [10]:
filter.drop(['PK'],1,inplace=True)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  """Entry point for launching an IPython kernel.


In [11]:
filter.dtypes

question_text     object
target           float64
dtype: object

In [12]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np

from keras.models import Sequential
from keras.callbacks import ModelCheckpoint
from keras.callbacks import EarlyStopping
from keras import models
from keras.models import load_model
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.layers import LSTM, Activation, Dense, Input, Embedding, Dropout
from keras.models import Model

from nltk import word_tokenize

from sklearn.metrics import roc_auc_score, accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.utils import class_weight

In [13]:
!wget http://nlp.stanford.edu/data/glove.42B.300d.zip

--2022-03-07 14:19:00--  http://nlp.stanford.edu/data/glove.42B.300d.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.42B.300d.zip [following]
--2022-03-07 14:19:00--  https://nlp.stanford.edu/data/glove.42B.300d.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/glove.42B.300d.zip [following]
--2022-03-07 14:19:01--  http://downloads.cs.stanford.edu/nlp/data/glove.42B.300d.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1877800501 (1.7G) [application/zip]
Sav

In [14]:
!unzip /content/glove.42B.300d.zip 

Archive:  /content/glove.42B.300d.zip
  inflating: glove.42B.300d.txt      


In [15]:
!rm /content/glove.42B.300d.zip

In [16]:
embeding_index={}

f=open('glove.42B.300d.txt',encoding='utf-8')

for line in f:
    values=line.split()
    word=values[0]
    coefs=np.asarray(values[1:],dtype='float32')
    embeding_index[word]=coefs
f.close()

In [17]:
len(embeding_index['provide'])

300

In [18]:
embeding_index['provide']

array([ 2.1902e-01, -6.6693e-02,  1.0538e-01, -2.0213e-01,  3.4797e-01,
       -8.4570e-03, -4.0350e+00,  3.4057e-01,  1.0484e-01,  2.1855e-01,
       -4.0837e-01,  2.7482e-01,  1.1953e-01,  7.4335e-02, -1.7069e-01,
       -4.0008e-01, -7.9120e-03, -3.9702e-01,  5.1159e-02, -2.3763e-02,
       -4.6144e-01,  4.7578e-02, -1.8110e-01,  3.9715e-03,  1.5718e-01,
       -1.8605e-01,  3.4680e-01,  1.7597e-01, -5.9390e-02, -3.0119e-01,
       -2.8326e-01, -2.6415e-01, -1.9358e-02,  6.7555e-03,  3.3796e-01,
       -3.5508e-01,  1.0336e-02, -5.0465e-01,  1.3932e-01, -2.7452e-01,
       -1.6377e-01,  5.0877e-01, -1.4303e-01, -2.0842e-01,  2.0463e-01,
        7.3660e-02, -1.8813e-01,  2.8356e-01, -2.7095e-01, -2.5136e-01,
       -1.2715e-01,  2.1164e-01, -4.4701e-01,  9.5515e-02,  4.5539e-01,
       -6.0036e-02,  1.4488e-02, -2.9433e-01, -5.5977e-01,  2.8814e-01,
        9.5750e-03,  2.2739e-01, -8.2802e-02,  3.0510e-01,  3.8501e-02,
        3.2902e-01, -1.1190e-02, -1.2909e-01,  3.6758e-01, -2.29

In [19]:
len(embeding_index)

1917494

In [20]:
X=filter['question_text']
Y=filter['target']

In [21]:
x_train,x_test,y_train,y_test=train_test_split(X,Y,test_size=0.2)
x_train.shape, x_test.shape, y_train.shape, y_test.shape

((1044897,), (261225,), (1044897,), (261225,))

In [34]:
#class_weight=class_weight.compute_class_weight('balanced', np.unique(y_train),y_train)
#print(class_weight)

TypeError: ignored

In [22]:
from sklearn.utils import compute_class_weight
class_weights = compute_class_weight(class_weight="balanced",classes=np.unique(y_train),y=y_train)
class_weight = dict(zip(np.unique(y_train), class_weights)),
class_weight

({0.0: 0.5330639352056861, 1.0: 8.061108453811853},)

In [23]:
class_weight_dict = dict(enumerate(class_weight))
class_weight_dict

{0: {0.0: 0.5330639352056861, 1.0: 8.061108453811853}}

In [24]:
x_train

908763     Who is the best online tutor for the preparati...
558560     Can a girl and boy be a life partner without g...
522560     Are special forces trained in intercepting ene...
403452      What is the level of JEE exam at a global level?
496556     Why do all other countries want to destroy Nor...
                                 ...                        
676448                 Does the Apple Newton count as a Mac?
316931     What marketing strategies are implemented to i...
903879     Can someone pursue a degree in international l...
824137                    Is it time for me to say good-bye?
1052722    If I were to wear a black and faded grey dress...
Name: question_text, Length: 1044897, dtype: object

In [25]:
from nltk import word_tokenize

In [26]:
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [27]:
sent_lens=[len(word_tokenize(x)) for x in x_train]
print(max(sent_lens))

412


In [28]:
np.percentile(sent_lens,99)

44.0

In [29]:
max_len=44

tk=Tokenizer(char_level=False,split=' ')

tk.fit_on_texts(x_train)

seq_train=tk.texts_to_sequences(x_train)
seq_test=tk.texts_to_sequences(x_test)

vocab_size=len(tk.word_index)

seq_train_matrix=sequence.pad_sequences(seq_train,maxlen=max_len)
seq_test_matrix=sequence.pad_sequences(seq_test,maxlen=max_len)

In [30]:
tk.word_index

{'the': 1,
 'what': 2,
 'is': 3,
 'a': 4,
 'to': 5,
 'in': 6,
 'of': 7,
 'i': 8,
 'how': 9,
 'and': 10,
 'do': 11,
 'are': 12,
 'for': 13,
 'you': 14,
 'can': 15,
 'why': 16,
 'it': 17,
 'my': 18,
 'that': 19,
 'if': 20,
 'with': 21,
 'on': 22,
 'or': 23,
 'have': 24,
 'be': 25,
 'does': 26,
 'from': 27,
 'your': 28,
 'an': 29,
 'which': 30,
 'should': 31,
 'get': 32,
 'when': 33,
 'best': 34,
 'would': 35,
 'as': 36,
 'people': 37,
 'some': 38,
 'there': 39,
 'who': 40,
 'will': 41,
 'like': 42,
 'at': 43,
 'not': 44,
 'about': 45,
 'by': 46,
 'they': 47,
 'was': 48,
 'did': 49,
 'any': 50,
 'we': 51,
 'so': 52,
 'good': 53,
 'me': 54,
 'their': 55,
 'one': 56,
 'india': 57,
 'has': 58,
 'after': 59,
 'most': 60,
 'where': 61,
 'make': 62,
 'this': 63,
 'but': 64,
 'more': 65,
 'all': 66,
 'think': 67,
 'many': 68,
 'between': 69,
 'time': 70,
 'than': 71,
 'much': 72,
 'other': 73,
 'life': 74,
 'someone': 75,
 'use': 76,
 'he': 77,
 'out': 78,
 'way': 79,
 'am': 80,
 'know': 81,
 'u

In [31]:
vocab_size

196330

In [32]:
seq_train_matrix.shape

(1044897, 44)

In [33]:
seq_train

[[40, 3, 1, 34, 151, 7191, 13, 1, 763, 7, 5939, 148, 856],
 [15, 4, 180, 10, 769, 25, 4, 74, 891, 94, 88132, 631],
 [12, 836, 1535, 4097, 6, 38366, 3196, 5786, 23, 3, 17, 96, 4, 290, 4482],
 [2, 3, 1, 414, 7, 311, 256, 43, 4, 938, 414],
 [16, 11, 66, 73, 192, 85, 5, 1717, 370, 583],
 [12,
  39,
  50,
  225,
  535,
  5,
  1,
  3764,
  22,
  184,
  4,
  153,
  447,
  19,
  48,
  3024,
  6,
  1,
  9582,
  23,
  10401],
 [16, 80, 8, 44, 176, 644, 650, 8, 106, 5, 1449, 780, 18, 935],
 [2, 11, 14, 67, 45, 2116, 19730, 2279, 11738, 2992],
 [9, 72, 3, 1, 309, 7, 187, 244, 220, 6, 205, 13, 2102, 418, 10, 3, 39, 5617],
 [9, 15, 14, 91, 1, 1022, 1426, 7, 4, 3717, 5679, 71, 446],
 [61,
  15,
  8,
  91,
  822,
  3296,
  13,
  29,
  4739,
  5,
  660,
  332,
  21,
  836,
  1145,
  1127,
  195,
  271,
  6184,
  3464,
  3179,
  46,
  64470,
  10,
  88133],
 [2, 3, 1, 34, 163, 13, 7786, 189, 6, 514],
 [2,
  3,
  1,
  2938,
  20,
  14,
  2368,
  4,
  822,
  7,
  580,
  3758,
  21247,
  21,
  394,
  509,


In [34]:
x_train.shape

(1044897,)

In [35]:
len(seq_train),len(seq_test)

(1044897, 261225)

In [36]:
seq_train_matrix.shape, seq_test_matrix.shape

((1044897, 44), (261225, 44))

In [37]:
embedding_matrix=np.zeros((vocab_size+1,300))

for word,i in tk.word_index.items():
    embed_vector=embeding_index.get(word)
    if embed_vector is not None:
        embedding_matrix[i]=embed_vector

In [38]:
 len(embedding_matrix)

196331

In [39]:
embedding_matrix.shape

(196331, 300)

In [40]:
embedding_matrix

array([[ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [-0.20838   , -0.14932001, -0.017528  , ..., -0.54066002,
         0.21199   , -0.0094357 ],
       [ 0.13877   , -0.0021201 , -0.091039  , ...,  0.024614  ,
         0.46514001,  0.11363   ],
       ...,
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ]])

In [41]:
inputs = Input(name='text_input',shape=[max_len])
embed=Embedding(vocab_size+1,300,input_length=max_len,mask_zero=True,
                weights=[embedding_matrix],trainable=False)(inputs)
lstm_layer1 = LSTM(512)(embed)
drop1 = Dropout(0.2)(lstm_layer1)
lstm_layer2 = Dense(256,activation='relu')(drop1)
drop2 = Dropout(0.2)(lstm_layer2)
lstm_layer3 = Dense(128,activation='relu')(drop2)
drop3 = Dropout(0.2)(lstm_layer3)
dense1 = Dense(25,activation='relu')(lstm_layer3)
drop4 = Dropout(0.2)(dense1)
final_layer = Dense(1,activation='sigmoid')(drop4)

model = Model(inputs=inputs,outputs=final_layer)

In [42]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 text_input (InputLayer)     [(None, 44)]              0         
                                                                 
 embedding (Embedding)       (None, 44, 300)           58899300  
                                                                 
 lstm (LSTM)                 (None, 512)               1665024   
                                                                 
 dropout (Dropout)           (None, 512)               0         
                                                                 
 dense (Dense)               (None, 256)               131328    
                                                                 
 dropout_1 (Dropout)         (None, 256)               0         
                                                                 
 dense_1 (Dense)             (None, 128)               32896 

In [43]:
filepath='/content/gdrive/weights-{epoch:02d}-{val_loss:.4f}.h5'
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, 
                             save_best_only=True)

In [88]:
filepath2='https://github.com/Shilpigoyal-hub/Quora-Spam-Filter-model/weights-{epoch:02d}-{val_loss:.4f}.h5'
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, 
                             save_best_only=True)

In [44]:
earlystop = EarlyStopping(monitor='val_loss', patience=3,
                          verbose=1)

In [45]:
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [46]:
model.fit(seq_train_matrix,y_train,
              epochs=15,
              class_weight={0: 0.5331085383324966, 1: 8.050922287457816},
              batch_size=1000,
              validation_data=(seq_test_matrix,y_test),
              callbacks = [earlystop, checkpoint])

Epoch 1/15
1045/1045 [==============================] - ETA: 0s - loss: 0.2962 - accuracy: 0.8734
Epoch 1: val_loss improved from inf to 0.26006, saving model to /content/gdrive/weights-01-0.2601.h5
1045/1045 [==============================] - 401s 375ms/step - loss: 0.2962 - accuracy: 0.8734 - val_loss: 0.2601 - val_accuracy: 0.8985
Epoch 2/15
1045/1045 [==============================] - ETA: 0s - loss: 0.2508 - accuracy: 0.8941
Epoch 2: val_loss did not improve from 0.26006
1045/1045 [==============================] - 384s 368ms/step - loss: 0.2508 - accuracy: 0.8941 - val_loss: 0.2782 - val_accuracy: 0.8866
Epoch 3/15
1045/1045 [==============================] - ETA: 0s - loss: 0.2274 - accuracy: 0.9016
Epoch 3: val_loss improved from 0.26006 to 0.24299, saving model to /content/gdrive/weights-03-0.2430.h5
1045/1045 [==============================] - 385s 368ms/step - loss: 0.2274 - accuracy: 0.9016 - val_loss: 0.2430 - val_accuracy: 0.9034
Epoch 4/15
1045/1045 [====================

In [50]:
best_model = load_model('/content/gdrive/weights-07-0.2065.h5')

In [86]:
from google.colab import drive
drive.mount('/content/gdrive')
#using model.save_weights() to save the weights of the model in HDF5 format
model.save_weights("/content/gdrive/My Drive/weights.h5")

Mounted at /content/gdrive


In [51]:
best_model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 text_input (InputLayer)     [(None, 44)]              0         
                                                                 
 embedding (Embedding)       (None, 44, 300)           58899300  
                                                                 
 lstm (LSTM)                 (None, 512)               1665024   
                                                                 
 dropout (Dropout)           (None, 512)               0         
                                                                 
 dense (Dense)               (None, 256)               131328    
                                                                 
 dropout_1 (Dropout)         (None, 256)               0         
                                                                 
 dense_1 (Dense)             (None, 128)               32896 

In [ ]:
p = best_model.predict(seq_test_matrix)
roc_auc_score(y_test,p)

In [ ]:
best_model.save('./spam_filter_model_quora_questions.h5')